In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.datasets import load_digits
skldata=load_digits()
skldf=pd.DataFrame(skldata["data"])
skldf.insert(0, 'target', skldata["target"])
skldf.head()

In [ ]:
data=np.array(skldf)
m,n=data.shape #m is examples n is features

dev_data=data[:200].T
Y_dev=dev_data[0]
X_dev=dev_data[1:n]
X_dev = X_dev / 255

train_data=data[200:m].T
Y_train = train_data[0]
X_train = train_data[1:n]
X_train=X_train/255


In [ ]:
def initialize_params():
    W1=np.random.rand(100,64) -0.5
    b1=np.random.rand(100,1)-0.5
    W2=np.random.rand(10,100) -0.5
    b2=np.random.rand(10,1)-0.5
    
    return W1,b1,W2,b2

def Relu(Z):
    return np.maximum(Z,0)

def Softmax(Z):
    return np.exp(Z) /sum(np.exp(Z))

def relu_der(Z):
    return Z>0

def one_hot(Y):
    one_hot_Y = pd.get_dummies(Y)
    one_hot_Y = one_hot_Y.values.T
    return one_hot_Y

def Forward_prop(W1,b1,W2,b2,X):
    Z1=W1.dot(X)+b1
    A1=Relu(Z1)
    Z2=W2.dot(A1)+b2
    A2=Softmax(Z2)
    return Z1,A1,Z2,A2


def back_prop(Z1,A1,Z2,A2,W1,W2,X,Y):
    one_hot_Y=one_hot(Y)
    dZ2=A2-one_hot_Y
    dW2=1/m * dZ2.dot(A1.T)
    db2=1/m * np.sum(dZ2)
    dZ1=W2.T.dot(dZ2)*relu_der(Z1)
    dW1=1/m * dZ1.dot(X.T)
    db1=1/m * np.sum(dZ1)
    return dW1,db1,dW2,db2

def update_params(W1,b1,W2,b2,dW1,db1,dW2,db2,alpha):
    W1=W1-alpha*dW1
    b1=b1-alpha*db1
    W2=W2-alpha*dW2
    b2=b2-alpha*db2
    return W1,b1,W2,b2

def predictions(A2):
    return np.argmax(A2,0)

def get_accuracy(predictions,Y):
    # print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

In [ ]:
def model(X,Y,alpha,epochs,threshold):
    W1,b1,W2,b2=initialize_params()
    for i in range(epochs):
        Z1,A1,Z2,A2=Forward_prop(W1,b1,W2,b2,X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        preds = predictions(A2)
        accuracy=get_accuracy(preds, Y)
        if i%10==0:
            print(f"Iteration: {i} \n Accuracy: {accuracy}")
        if accuracy>=threshold:
            return W1, b1, W2, b2
        
    return W1, b1, W2, b2



In [ ]:
W1, b1, W2, b2 = model(X_train, Y_train, 0.30, 500,.80)

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = Forward_prop(W1, b1, W2, b2, X)
    return predictions(A2)

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((8, 8)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(9, W1, b1, W2, b2)